In [ ]:
 select distinct visit_id,  count_days from (
    select visit_id,
           visit_date,
           count(*) over(partition by visit_id order by visit_date range between 1  preceding  and 1 following  )  count_days
    from orders group by  visit_id, visit_date
)
where count_days = 3;

In [ ]:
WITH t1 AS (
    SELECT v1.customer_id, v1.visit_date, v1.amount,
        SUM(v2.amount) OVER (PARTITION BY v1.customer_id ORDER BY v1.visit_date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS rolling_sum
    FROM visits v1
    JOIN visits v2 ON v1.customer_id = v2.customer_id AND v2.visit_date BETWEEN DATEADD(day, -4, v1.visit_date) AND v1.visit_date
), t2 AS (
    SELECT customer_id, rolling_sum,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY rolling_sum DESC) AS rn
    FROM t1
)
SELECT customer_id, rolling_sum AS max_amount
FROM t2
WHERE rn = 1;

In [ ]:
WITH t1 AS (
    SELECT a.airplane_id, a.airline_id, a.total_seats, 
           COALESCE(SUM(b.booked), 0) AS booked_seats,
           a.total_seats - COALESCE(SUM(b.booked), 0) AS emp_seats
    FROM airlines_detail a
    LEFT JOIN bookings b ON a.airplane_id = b.airplane_id
    GROUP BY a.airplane_id, a.airline_id, a.total_seats
),
t2 AS (
    SELECT airline_id, AVG(emp_seats) AS t2
    FROM t1
    GROUP BY airline_id
),
t3 AS (
    SELECT t1.airplane_id, t1.airline_id, t1.emp_seats, 
           RANK() OVER (PARTITION BY t1.airline_id ORDER BY ABS(t1.emp_seats - t2.t2)) AS rank
    FROM t1 t1
    JOIN t2 ON t1.airline_id = t2.airline_id
)
SELECT airline_id, STRING_AGG(airplane_id, ',') AS airplanes
FROM t3
WHERE rank = 1
GROUP BY airline_id
ORDER BY airline_id;